In [1]:
with open("resources/train_goldDDI.txt","r") as fp:
    lines = fp.readlines()
ents = [line.replace("\n","").split("|") for line in lines]
ents

[['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e0',
  'DDI-DrugBank.d542.s0.e1',
  '0',
  'null'],
 ['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e0',
  'DDI-DrugBank.d542.s0.e2',
  '0',
  'null'],
 ['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e0',
  'DDI-DrugBank.d542.s0.e3',
  '0',
  'null'],
 ['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e0',
  'DDI-DrugBank.d542.s0.e4',
  '0',
  'null'],
 ['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e1',
  'DDI-DrugBank.d542.s0.e2',
  '0',
  'null'],
 ['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e1',
  'DDI-DrugBank.d542.s0.e3',
  '0',
  'null'],
 ['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e1',
  'DDI-DrugBank.d542.s0.e4',
  '0',
  'null'],
 ['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e2',
  'DDI-DrugBank.d542.s0.e3',
  '0',
  'null'],
 ['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e2',
  'DDI-DrugBank.d542.s0.e4',
  '0',
  'null'],
 ['DDI-DrugBank.d542.s0',
  'DDI-DrugBank.d542.s0.e3',
  'DDI-DrugBank.d542.s0.e4'

In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame(ents, columns=["sentence","e1","e2","is_DDI","DDI_type"])
print(df.shape)
df

(25089, 5)


,sentence,e1,e2,is_DDI,DDI_type
0,DDI-DrugBank.d542.s0,DDI-DrugBank.d542.s0.e0,DDI-DrugBank.d542.s0.e1,0,null
1,DDI-DrugBank.d542.s0,DDI-DrugBank.d542.s0.e0,DDI-DrugBank.d542.s0.e2,0,null
2,DDI-DrugBank.d542.s0,DDI-DrugBank.d542.s0.e0,DDI-DrugBank.d542.s0.e3,0,null
3,DDI-DrugBank.d542.s0,DDI-DrugBank.d542.s0.e0,DDI-DrugBank.d542.s0.e4,0,null
4,DDI-DrugBank.d542.s0,DDI-DrugBank.d542.s0.e1,DDI-DrugBank.d542.s0.e2,0,null
...,...,...,...,...,...
25084,DDI-MedLine.d106.s8,DDI-MedLine.d106.s8.e0,DDI-MedLine.d106.s8.e2,1,int
25085,DDI-MedLine.d106.s8,DDI-MedLine.d106.s8.e1,DDI-MedLine.d106.s8.e2,1,int
25086,DDI-MedLine.d106.s9,DDI-MedLine.d106.s9.e0,DDI-MedLine.d106.s9.e1,0,null
25087,DDI-MedLine.d106.s9,DDI-MedLine.d106.s9.e0,DDI-MedLine.d106.s9.e2,0,null


In [5]:
df["DDI_type"].unique()

array(['null', 'effect', 'mechanism', 'advise', 'int'], dtype=object)